In [45]:
import math
import json
from pathlib import Path
import pickle
import time

from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import httpx

# for i in range(10):
#     clear_output(wait=True)
#     print(i)
#     time.sleep(0.1)

DATA_DIR = Path('./data')
WORKS_DATA_DIR = DATA_DIR / 'works_of_institutions'
AUTHORS_DIR = DATA_DIR / 'works_by_author'
print(len(list(WORKS_DATA_DIR.glob('*.json'))))

1134


## institutional works to author_id

In [28]:
work_file = WORKS_DATA_DIR / 'work-2020-true-1.json'

with work_file.open('r', encoding='UTF-8') as infile:
    work_json = json.load(infile)
print(len(work_json['results']))

print(work_json['results'][0].keys())
for r in work_json['results']:
    clear_output(wait=True)
    print(r['id'])
    matched_authors = [x for x in r['authorships'] if any(i for i in x['institutions'] if i['id'] in SAMPLE_INSTITUTIONS)]
    for auth in matched_authors:
        print(auth['author']['id'], auth['author']['display_name'], [i['display_name'] for i in auth['institutions']])
    print()
    


https://openalex.org/W3087139986
https://openalex.org/A2814149936 Yuzi Tian ['University of Michigan–Ann Arbor', 'Department of Rheumatology and Immunology, Xiangya Hospital, Central South University, Changsha, Hunan, China.']
https://openalex.org/A2607418774 Qiufang Deng ['University of Michigan–Ann Arbor', 'Department of Rheumatology and Immunology, Xiangya Hospital, Central South University, Changsha, Hunan, China.']
https://openalex.org/A2274029971 Hasan B. Alam ['University of Michigan–Ann Arbor']
https://openalex.org/A2107458222 Yongqing Li ['University of Michigan–Ann Arbor']



In [13]:
SAMPLE_INSTITUTIONS = (
    'https://openalex.org/I27837315',
    'https://openalex.org/I4210114445',
    'https://openalex.org/I4210151884'
)

In [ ]:
author_ids = set()

for work_file in WORKS_DATA_DIR.glob("work-*.json"):
    clear_output(wait=True)
    print(work_file)
    with work_file.open('r', encoding='UTF-8') as infile:
        work_json = json.load(infile)
    for r in work_json['results']:
        print(r['id'])
        matched_authors = [x for x in r['authorships'] if any(i for i in x['institutions'] if i.get('id') in SAMPLE_INSTITUTIONS)]
        for auth in matched_authors:
            author_id = auth['author'].get('id')
            if author_id and author_id not in author_ids:
                author_ids.add(auth['author']['id'])
                print(author_id, auth['author']['display_name'], [i.get('display_name') for i in auth['institutions']])
    print('up to', len(author_ids))
print('done')

In [38]:
authors_out = WORKS_DATA_DIR / 'authors_from_works_of_institutions.json'
with authors_out.open('w', encoding='UTF-8') as outfile:
    json.dump(list(author_ids), outfile)
print(len(author_ids))

99174


### author_id to authored works

In [75]:
CONTACT_EMAIL = "matvan@umich.edu"
# A2669589972 was a problem, very large file to load (??), timing out on httpx defaults
PER_PAGE = 200

In [52]:
with authors_out.open('r', encoding='UTF-8') as infile:
    authors = list(json.load(infile))
len(authors)

99174

In [ ]:
for full_author_id in authors:
    clear_output(wait=True)
    author_id = full_author_id.replace('https://openalex.org/', '')
    print(author_id, i)
    # for year in range(MAX_YEAR, MIN_YEAR-1, -1):
    # filtering = f"filter=author.id:{author_id},publication_year:{year}"
    base_url = f"https://api.openalex.org/works?mailto={CONTACT_EMAIL}"
    filtering = f"filter=author.id:{author_id}"
    for i in range(1, 1000):
        pagination = f"per_page={PER_PAGE}&page={i}"
        url = "&".join((base_url, filtering, pagination))
        print(url)

        filename = f"{author_id}-{i:0>3}.json"
        path = AUTHORS_DIR / filename
        if path.exists():
            print(f"Skipping {path}")
            continue

        try:
            j = httpx.get(url, timeout=30.0).json()
        except Exception as e:
            error_path = DATA_DIR / f"ERROR-{filename}"
            error_path.write_text(url + "\n\n" + str(e))
            continue
        print(j['meta'])

        count = j['meta']['count']
        page = j['meta']['page']
        per_page = j['meta']['per_page']

        if j['results']:
            print(path)
            with path.open('w', encoding='UTF-8') as outfile:
                json.dump(j, outfile)
        time.sleep(1)
        if per_page * page >= count:
            break

A2470637351 2
https://api.openalex.org/works?mailto=matvan@umich.edu&filter=author.id:A2470637351&per_page=200&page=1
Skipping data\works_by_author\A2470637351-001.json
https://api.openalex.org/works?mailto=matvan@umich.edu&filter=author.id:A2470637351&per_page=200&page=2
{'count': 8, 'db_response_time_ms': 14, 'page': 2, 'per_page': 200}
